In [86]:
import pinocchio as pin
import numpy as np

robot = pin.RobotWrapper()
robot.initFromURDF("Exo/description_v1.urdf")

print('model name: ' + robot.model.name)
 
# Sample a random configuration
q        = pin.randomConfiguration(robot.model)
vq        = pin.randomConfiguration(robot.model)
aq0 = pin.randomConfiguration(robot.model)

robot.forwardKinematics(q)

model name: EXO_TAO


In [98]:
robot.nle(q,vq)
robot.mass(q)
robot.computeJointJacobians(q)



# print("M: ")
# print(data.M)

# print("Aq: ")
# print(data.Ag)

# print("J: ")
# print(data.J)

# print("G: ")
# print(data.g)


print( robot.data.M )
print( -np.linalg.inv(robot.data.M) )
print(robot.data.C)


[[0.3764 0.1    0.     0.    ]
 [0.1    0.1    0.     0.    ]
 [0.     0.     0.3764 0.1   ]
 [0.     0.     0.1    0.1   ]]
[[ -3.61794501   3.61794501  -0.          -0.        ]
 [  3.61794501 -13.61794501  -0.          -0.        ]
 [ -0.          -0.          -3.61794501   3.61794501]
 [ -0.          -0.           3.61794501 -13.61794501]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [103]:
import sys
import os

from pinocchio.visualize import GepettoVisualizer
viz = GepettoVisualizer(robot.model)

try:
    viz.initViewer()
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install gepetto-viewer")
    print(err)
    sys.exit(0)

/tmp/ipykernel_4418/1210759568.py:8: UserWarning: Error while importing the viewer client.
Check whether gepetto-gui is properly installed
  viz.initViewer()
/tmp/ipykernel_4418/1210759568.py:8: UserWarning: Error while starting the viewer client.
Check whether gepetto-viewer is properly started
  viz.initViewer()


In [170]:
from panda3d_viewer import ViewerClosedError
from pinocchio.visualize.panda3d_visualizer import Panda3dVisualizer

robot.setVisualizer(Panda3dVisualizer())
robot.initViewer()
robot.loadViewerModel(group_name='EXO_TAO', color=(1, 1, 1, 1))

ts = 1/200

t = np.linspace(0,10,int(10/ts))
traj = np.empty((4,t.shape[0]))

for i,t_cur in enumerate(t):
    traj[0][i] = np.sin(2*np.pi*t_cur)
    traj[1][i] = np.sin(2*np.pi*t_cur*2)
    traj[2][i] = np.sin(2*np.pi*t_cur)
    traj[3][i] = np.sin(2*np.pi*t_cur*2)

    # robot.display(traj[:,i])

robot.play(traj[:,::2],1/100)


Xlib:  extension "XFree86-DGA" missing on display ":0".


ViewerClosedError: User closed the main window

In [167]:
10/ts


2000.0